# Qwen3-14B Jordan Peterson Fine-Tuning — Version 3
## Cleaner Q&A Data · 3 Epochs · r=32 LoRA

This notebook fine-tunes Qwen3-14B on Jordan B. Peterson's four books using
Version 3 of the training pipeline.

**Pipeline position:** Run `JordanPeterson_DataPrep.ipynb` first to generate
the Q&A cache (`qa_dataset/peterson_qa.jsonl`).  This notebook reads from that
cache directly — no PDF extraction or question generation happens here.

**V3 = fine-tuning only.**  The dataset generation that was embedded in V2
(Part 1) has been extracted into the standalone DataPrep notebook.  V3 trains
on the same Haiku-generated Q&A pairs but with front-matter removal applied,
producing a cleaner dataset (4,867 pairs instead of V2's 5,029).

---
## Version History

| Component | V1 | V2 | V3 |
|-----------|----|----|-----|
| Training task | Passage completion | Synthetic Q&A | Synthetic Q&A |
| Dataset | Raw PDF text (with front matter) | Haiku Q&A (with front matter) | Haiku Q&A (clean — no front matter) |
| Passages | ~2,519 (all pages) | ~2,519 (all pages) | ~2,492 (front-matter removed) |
| Q&A pairs | N/A | 5,029 | 4,867 (Haiku, front-matter-free) |
| Epochs | 1 | 3 | 3 |
| LoRA rank | r=16 | r=32 | r=32 |
| LoRA alpha | 16 | 32 | 32 |
| Output adapter | `qwen3_14b_…_v1_lora` | `qwen3_14b_peterson_v2_lora` | `qwen3_14b_peterson_v3_lora` |

V3 uses the same hyperparameters as V2.  The only change is the dataset: the
DataPrep notebook's improved front-matter removal is now applied consistently.

---
## Why V3 Data Is Better Than V2's

The V2 dataset was generated from the same extraction pass used in V1, which
included front-matter pages.  The DataPrep notebook introduced a 3-tier
heuristic to skip publisher pages before Chapter 1.  V3 trains on the output
of that improved extraction.

### Maps of Meaning — the biggest improvement

V2 had **1,826 pairs** including title pages, the ROUTLEDGE copyright block,
and the full table of contents from page 0.  V3 starts from page 7 (the
actual philosophical content), dropping those pages — fewer pairs but all
substantive.

Without the front-matter fix, the very first training example paired a
philosophical question with an answer that began:

> "MAPS OF MEANING MAPS OF MEANING The Architecture of Belief JORDAN B.PETERSON
> ROUTLEDGE New York and London Published in 1999 by Routledge…"

The model was learning to associate philosophical questions with publisher
metadata.

### We Who Wrestle with God

V2 had **1,367 pairs** starting from the Penguin copyright page ("Penguin
Random House values and supports copyright…").  V3 skips those pages and
starts from actual content.

### Other books

12 Rules for Life and Beyond Order saw minimal change (4–5% reduction) —
their front matter was shorter and the fix only removed a few boilerplate
pages.

In [1]:
import json, re, math, time, gc
from pathlib import Path

import torch
from datasets import Dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer, SFTConfig
from unsloth.chat_templates import train_on_responses_only

# ── Load shared config ─────────────────────────────────────────────────────
# All paths and the system prompt are centralised in peterson_config.json so
# they stay in sync across the DataPrep and fine-tuning notebooks.
with open("peterson_config.json") as f:
    _config = json.load(f)

QA_CACHE      = Path(_config["paths"]["qa_cache"])
SYSTEM_PROMPT = _config["system_prompt"]

# ── V3 model + training constants ─────────────────────────────────────────
BASE_MODEL    = "unsloth/Qwen3-14B-unsloth-bnb-4bit"
OUTPUT_DIR    = Path("./outputs/qwen3_14b_peterson_v3_lora")   # does NOT touch V2
MAX_SEQ_LEN   = 2048
LORA_RANK     = 32
LORA_ALPHA    = 32
BATCH_SIZE    = 2
GRAD_ACCUM    = 4
NUM_EPOCHS    = 3
LEARNING_RATE = 2e-4
WARMUP_STEPS  = 30
WEIGHT_DECAY  = 0.01

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"PyTorch  : {torch.__version__}")
print(f"CUDA     : {torch.cuda.is_available()}  |  GPU: {torch.cuda.get_device_name(0)}")
print(f"VRAM     : {torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB")
print()
print("Configuration:")
print(f"  Base model  : {BASE_MODEL}")
print(f"  LoRA rank   : r={LORA_RANK}, alpha={LORA_ALPHA}  (ratio={LORA_ALPHA/LORA_RANK:.1f})")
print(f"  Epochs      : {NUM_EPOCHS}")
print(f"  Batch (eff) : {BATCH_SIZE} x {GRAD_ACCUM} = {BATCH_SIZE * GRAD_ACCUM}")
print(f"  LR          : {LEARNING_RATE}")
print(f"  QA cache    : {QA_CACHE}")
print(f"  Output      : {OUTPUT_DIR.resolve()}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
PyTorch  : 2.10.0+cu128
CUDA     : True  |  GPU: NVIDIA GeForce RTX 4090
VRAM     : 25.3 GB

Configuration:
  Base model  : unsloth/Qwen3-14B-unsloth-bnb-4bit
  LoRA rank   : r=32, alpha=32  (ratio=1.0)
  Epochs      : 3
  Batch (eff) : 2 x 4 = 8
  LR          : 0.0002
  QA cache    : qa_dataset/peterson_qa.jsonl
  Output      : /home/rob/PythonEnvironments/FineTuning/FineTuning/NoteBooks/JordanPeterson/outputs/qwen3_14b_peterson_v3_lora


---
# Part 1: Load the Q&A Dataset

The `JordanPeterson_DataPrep.ipynb` notebook generated this cache by:
1. Extracting ~350-word passages from each PDF with front-matter removal
2. Calling Claude Haiku to generate 2 questions per passage

V2 trained on **5,029 pairs** generated from ~2,519 passages (front matter
included).  V3 trains on **4,867 pairs** from ~2,492 passages — the 27 pairs
of reduction come entirely from dropping publisher metadata pages, not from
removing real philosophical content.

In [2]:
from collections import Counter

if not QA_CACHE.exists():
    raise FileNotFoundError(
        f"Q&A cache not found at {QA_CACHE}. "
        "Run JordanPeterson_DataPrep.ipynb first to generate the dataset."
    )

with open(QA_CACHE) as f:
    records = [json.loads(line) for line in f if line.strip()]

# Build a HuggingFace Dataset from the flat list of dicts.
# We only keep "question" and "answer"; the "book" field is used for statistics.
raw_dataset = Dataset.from_list([
    {"question": r["question"], "answer": r["answer"]}
    for r in records
])

print(f"Dataset loaded: {len(raw_dataset):,} Q&A pairs")
print(f"Schema: {raw_dataset.column_names}")
print()

book_dist = Counter(r["book"] for r in records)
print("Distribution by book:")
for book, count in sorted(book_dist.items(), key=lambda x: -x[1]):
    print(f"  {book:<35}  {count:4d}  ({100*count/len(records):.1f}%)")

# V2 reference distribution for comparison
print()
print("V2 distribution (for comparison):")
v2_ref = {
    "Maps of Meaning": 1826,
    "We Who Wrestle with God": 1367,
    "12 Rules for Life": 974,
    "Beyond Order": 862,
}
for book, count in sorted(v2_ref.items(), key=lambda x: -x[1]):
    v3_count = book_dist.get(book, 0)
    delta = v3_count - count
    print(f"  {book:<35}  V2: {count:4d}  →  V3: {v3_count:4d}  ({delta:+d})")

Dataset loaded: 4,867 Q&A pairs
Schema: ['question', 'answer']

Distribution by book:
  Maps of Meaning                      1821  (37.4%)
  We Who Wrestle with God              1264  (26.0%)
  12 Rules for Life                     926  (19.0%)
  Beyond Order                          856  (17.6%)

V2 distribution (for comparison):
  Maps of Meaning                      V2: 1826  →  V3: 1821  (-5)
  We Who Wrestle with God              V2: 1367  →  V3: 1264  (-103)
  12 Rules for Life                    V2:  974  →  V3:  926  (-48)
  Beyond Order                         V2:  862  →  V3:  856  (-6)


---
# Part 2: Load the Qwen3-14B Base Model

## Why Qwen3-14B Over GPT-OSS 20B

Both models were evaluated in V1.  Qwen3-14B is the better choice for three
independent reasons:

1. **Lower training loss** (2.44 vs 3.01) — despite being a smaller model,
   Qwen3 absorbed the domain signal more efficiently in one epoch.

2. **Training speed** (23 min vs 73 min per epoch) — with 3 epochs planned,
   GPT-OSS would take ~220 min; Qwen3 takes ~70 min.

3. **Architectural recency** — Qwen3-14B (2025) is a more capable base model
   for instruction-following, which matters for the Q&A format we train on.

## Why `max_seq_length=2048`

Our training examples are ~350-word passages formatted as ChatML conversations.
After tokenisation, each example is typically 650–800 tokens.  Setting
`max_seq_length=2048` gives headroom for the system prompt and formatting
overhead without wasting VRAM on unnecessary KV-cache pre-allocation.

In [3]:
print(f"Loading {BASE_MODEL} ...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name      = BASE_MODEL,
    dtype           = None,          # auto: bfloat16 on Ampere+, float16 otherwise
    max_seq_length  = MAX_SEQ_LEN,
    load_in_4bit    = True,          # 4-bit quantization via bitsandbytes
    full_finetuning = False,         # we are adding LoRA, not full fine-tuning
)

vram_after_load = torch.cuda.memory_reserved() / 1e9
print(f"Model loaded.  VRAM reserved: {vram_after_load:.1f} GB")
print(f"Model dtype   : {next(model.parameters()).dtype}")
print(f"Tokenizer     : {tokenizer.__class__.__name__}")

Loading unsloth/Qwen3-14B-unsloth-bnb-4bit ...
==((====))==  Unsloth 2026.2.1: Fast Qwen3 patching. Transformers: 4.57.6.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.516 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded.  VRAM reserved: 11.2 GB
Model dtype   : torch.bfloat16
Tokenizer     : Qwen2TokenizerFast


---
# Part 3: Add LoRA Adapters (r=32)

## What LoRA Does

Low-Rank Adaptation freezes the pre-trained weight matrices `W` and injects a
trainable update `ΔW = (alpha/r) × B × A` where:
- `B` has shape `(d_model × r)` — initialised to zero
- `A` has shape `(r × d_model)` — initialised with random Gaussian values
- `r` is the rank

At the start of training `B=0` so `ΔW=0` and the model behaves identically
to the base model.  Training gradually fills `B` with the directions needed
to capture Peterson's style.

## Why r=32 vs V1's r=16

Think of `r` as the number of "stylistic dimensions" the adapter can learn.
With r=16, the adapter has 16 orthogonal directions to encode things like:
*"use chaos/order vocabulary"*, *"frame arguments historically"*,
*"connect psychology to mythology"*, etc.

With r=32 we have 32 such dimensions — enough to capture both the macro-level
thematic vocabulary AND the micro-level sentence rhythm that makes Peterson's
writing recognisable.  The adapter checkpoint grows from ~260 MB to ~520 MB;
VRAM impact is negligible.

## Target Modules

We apply LoRA to all attention projections (q, k, v, o) and all MLP
gate/up/down projections.  This is the full set recommended by Unsloth for
stylistic fine-tuning because style is encoded across both attention patterns
(what the model attends to) and MLP activations (how it transforms those
representations).

Setting `alpha = rank` (alpha=32, rank=32) keeps the effective scale factor
at `alpha/rank = 1.0` — the LoRA update is applied at the same relative
strength as the base weights, which is the established baseline.

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r                   = LORA_RANK,    # 32 — doubled from V1's r=16
    lora_alpha          = LORA_ALPHA,   # 32 — alpha/rank = 1.0 (same scale as V1)
    lora_dropout        = 0,            # 0 is optimal for Unsloth according to docs
    target_modules      = [
        "q_proj", "k_proj", "v_proj", "o_proj",   # all attention projections
        "gate_proj", "up_proj", "down_proj",        # all MLP projections
    ],
    use_gradient_checkpointing = "unsloth",   # saves ~30% VRAM vs PyTorch default
    random_state        = 42,
    use_rslora          = False,   # standard LoRA (not rank-stabilised variant)
    loftq_config        = None,    # no quantization-aware init needed with 4-bit base
)

# ── Count trainable parameters ─────────────────────────────────────────────
total_params     = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pct_trainable    = 100 * trainable_params / total_params

print(f"Total parameters    : {total_params:>15,}")
print(f"Trainable (LoRA)    : {trainable_params:>15,}  ({pct_trainable:.4f}% of total)")
print(f"  V1 had r=16: ~{trainable_params//2:,} trainable params")
print(f"  V3 has r=32: ~{trainable_params:,} trainable params (2x capacity)")
print(f"VRAM after LoRA     : {torch.cuda.memory_reserved()/1e9:.1f} GB")

Unsloth 2026.2.1 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


Total parameters    :   8,691,809,280
Trainable (LoRA)    :     128,450,560  (1.4778% of total)
  V1 had r=16: ~64,225,280 trainable params
  V3 has r=32: ~128,450,560 trainable params (2x capacity)
VRAM after LoRA     : 11.7 GB


---
# Part 4: Format Dataset into Qwen3 ChatML Format

Each Q&A pair is converted into a three-turn conversation using the Qwen3
ChatML template.  The format change from V1 to V2/V3:

| Role | V1 content | **V2/V3 content** |
|------|-----------|----------------|
| `system` | Generic assistant prompt | **Peterson persona prompt** |
| `user` | A book passage chunk | **A synthetically generated question** |
| `assistant` | Continuation of the passage | **The original passage as an answer** |

The system prompt changes from generic to persona-specific so the model
learns: *"when given the Peterson persona instruction, respond in his voice"*.

The user message changes from a passage fragment to a real question so the
model learns: *"when asked a question in this domain, produce a substantive
Peterson-style answer"*.

## About `enable_thinking=False`

We format training examples with `enable_thinking=False`.  This tells Qwen3's
chat template not to include chain-of-thought reasoning tokens in the formatted
string.  Even with this setting, the template adds an empty
`<think>\n\n</think>` block before the assistant response — this is expected
and will be masked out by `train_on_responses_only` in the next step, so it
does not contribute to the loss.

## System Prompt Source

`SYSTEM_PROMPT` is loaded from `peterson_config.json` (not hardcoded) so
the DataPrep notebook and all fine-tuning notebooks use identical persona
instructions.

In [5]:
def format_example(batch):
    '''
    Convert a batch of Q&A records into formatted Qwen3 ChatML strings.

    Each formatted string looks like:
        <|im_start|>system
        You are an AI assistant... (Peterson persona)
        <|im_end|>
        <|im_start|>user
        <the generated question>
        <|im_end|>
        <|im_start|>assistant
        <think>

        </think>
        <the Peterson passage>
        <|im_end|>

    The <think></think> block appears because Qwen3's template always includes
    a thinking placeholder even when enable_thinking=False.  It is masked out
    during training by train_on_responses_only and does not affect the loss.
    '''
    formatted_texts = []
    for question, answer in zip(batch["question"], batch["answer"]):
        conversation = [
            {"role": "system",    "content": SYSTEM_PROMPT},
            {"role": "user",      "content": question},
            {"role": "assistant", "content": answer},
        ]
        text = tokenizer.apply_chat_template(
            conversation,
            tokenize              = False,
            add_generation_prompt = False,
            enable_thinking       = False,
        )
        formatted_texts.append(text)
    return {"text": formatted_texts}


dataset = raw_dataset.map(
    format_example,
    batched=True,
    batch_size=256,
    remove_columns=raw_dataset.column_names,
)

print(f"Formatted dataset: {len(dataset):,} examples")
print(f"Columns: {dataset.column_names}")
print()
print("-" * 70)
print("SAMPLE FORMATTED TRAINING EXAMPLE")
print("-" * 70)
sample = dataset[0]["text"]
print(sample[:800])
print("...")
print("-" * 70)
print(f"Full sample length: {len(sample.split())} words / {len(sample)} chars")

Map:   0%|          | 0/4867 [00:00<?, ? examples/s]

Formatted dataset: 4,867 examples
Columns: ['text']

----------------------------------------------------------------------
SAMPLE FORMATTED TRAINING EXAMPLE
----------------------------------------------------------------------
<|im_start|>system
You are an AI assistant that has been trained on the complete works of Jordan B. Peterson, a Canadian clinical psychologist, professor, and author. You speak with deep knowledge of psychology, philosophy, mythology, religion, and personal responsibility. Your responses reflect Peterson's writing style, intellectual depth, and interdisciplinary approach to understanding human nature and meaning.<|im_end|>
<|im_start|>user
How does the brain's neurological structure relate to our psychological capacity to generate meaning from chaos?<|im_end|>
<|im_start|>assistant
<think>

</think>

FIGURES 1 The Domain and Constituent Elements of the Known 15 2 The Metamythological Cycle of the Way 17 3 Normal Life 28 4 Revolutionary Adaptation 31 5 The Ambiv

In [6]:
# Auto-detect the response boundary tokens by inspecting the formatted data.
# This guards against future Qwen3 template changes that might shift the token
# boundaries, and mirrors the robust approach used in V2.
_sample_text = dataset[0]["text"]

if "<|im_start|>assistant\n" in _sample_text:
    instruction_part = "<|im_start|>user\n"
    response_part    = "<|im_start|>assistant\n"
    print("Detected Qwen3 ChatML response boundary tokens:")
else:
    raise RuntimeError(
        "Could not detect Qwen3 ChatML tokens in formatted dataset. "
        "Check the output of format_example() above."
    )

print(f"  instruction_part : {repr(instruction_part)}")
print(f"  response_part    : {repr(response_part)}")
print()
print("Boundary tokens set — train_on_responses_only will be applied below.")

Detected Qwen3 ChatML response boundary tokens:
  instruction_part : '<|im_start|>user\n'
  response_part    : '<|im_start|>assistant\n'

Boundary tokens set — train_on_responses_only will be applied below.


---
# Part 5: Configure and Run Training

## SFTConfig Parameters Explained

- **`num_train_epochs=3`**: The most important change from V1.  Three full
  passes over the dataset let the model move from memorisation into genuine
  stylistic generalisation.  The empirical sweet spot for LoRA stylistic
  fine-tuning: 1 epoch = memorisation, 3 epochs = generalisation, 5+ = risk
  of overfitting.

- **`warmup_steps=30`**: Gradually ramps the learning rate from 0 → 2e-4
  over the first 30 gradient steps (~3% of total training).  Prevents the
  randomly-initialised LoRA matrices from making destructively large weight
  updates at the start.

- **`lr_scheduler_type="cosine"`**: After warmup the learning rate decays
  smoothly following a cosine curve, reaching ~0 at the end of epoch 3.
  Cosine decay is generally more stable than linear decay for LoRA.

- **`weight_decay=0.01`**: Mild L2 regularisation on the LoRA weights.
  Discourages the adapter from memorising individual passages and encourages
  generalisation.

- **`optim="adamw_8bit"`**: Unsloth's 8-bit AdamW keeps optimizer states in
  8-bit rather than 32-bit, saving ~3 GB of VRAM with no measurable quality
  cost for LoRA training.

- **`packing=False`**: Sequence packing combines multiple short examples into
  one long sequence for efficiency.  We disable it because our examples are
  long (~400 words → ~700 tokens), and packing them could leak context between
  unrelated Q&A pairs.

## V3 Expected Step Count

With 4,867 pairs (vs V2's 5,029), the expected gradient updates are:

```
ceil(4867 / 2) × 3 // 4 = 2434 × 3 // 4 = 7302 // 4 = 1,825
```

V2 ran **1,887 steps** from 5,029 pairs — V3 should run approximately
**1,825 steps**, a minor reduction from cleaner data.

In [7]:
sft_trainer = SFTTrainer(
    model            = model,
    processing_class = tokenizer,
    train_dataset    = dataset,
    args             = SFTConfig(
        dataset_text_field          = "text",
        max_length                  = MAX_SEQ_LEN,
        dataset_num_proc            = 2,

        per_device_train_batch_size = BATCH_SIZE,
        gradient_accumulation_steps = GRAD_ACCUM,
        num_train_epochs            = NUM_EPOCHS,

        learning_rate               = LEARNING_RATE,
        warmup_steps                = WARMUP_STEPS,
        lr_scheduler_type           = "cosine",
        weight_decay                = WEIGHT_DECAY,

        optim                       = "adamw_8bit",
        fp16                        = not torch.cuda.is_bf16_supported(),
        bf16                        = torch.cuda.is_bf16_supported(),

        logging_steps               = 25,
        save_strategy               = "epoch",
        output_dir                  = str(OUTPUT_DIR),
        report_to                   = "none",

        seed                        = 42,
        packing                     = False,
    ),
)

sft_trainer = train_on_responses_only(
    sft_trainer,
    instruction_part = instruction_part,
    response_part    = response_part,
)

# ── Masking verification ───────────────────────────────────────────────────
# If masking is working, labels will be -100 for all system/user tokens and
# real token IDs only for the assistant response tokens.
_sample_input = sft_trainer.train_dataset[0]
_n_total      = len(_sample_input["input_ids"])
_n_trained    = sum(1 for lbl in _sample_input["labels"] if lbl != -100)
_n_masked     = _n_total - _n_trained
print("Response masking check on first example:")
print(f"  Total tokens   : {_n_total}")
print(f"  Trained tokens : {_n_trained}  (assistant response only)")
print(f"  Masked tokens  : {_n_masked}   (system + user = {100*_n_masked/_n_total:.0f}% of input)")
print()

_total_steps = math.ceil(len(dataset) / BATCH_SIZE) * NUM_EPOCHS // GRAD_ACCUM
print(f"Estimated gradient updates: {_total_steps:,}")
print(f"  ({len(dataset):,} examples / {BATCH_SIZE} batch x {GRAD_ACCUM} accum x {NUM_EPOCHS} epochs)")

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/4867 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


Map (num_proc=20):   0%|          | 0/4867 [00:00<?, ? examples/s]

Filter (num_proc=20):   0%|          | 0/4867 [00:00<?, ? examples/s]

Response masking check on first example:
  Total tokens   : 744
  Trained tokens : 645  (assistant response only)
  Masked tokens  : 99   (system + user = 13% of input)

Estimated gradient updates: 1,825
  (4,867 examples / 2 batch x 4 accum x 3 epochs)


In [ ]:
import time

vram_before = torch.cuda.memory_reserved() / 1e9
print(f"VRAM before training: {vram_before:.1f} GB")
print(f"Starting training — {NUM_EPOCHS} epochs, ~{_total_steps:,} gradient updates...")
print()

t0 = time.time()
train_result = sft_trainer.train()
elapsed_min  = (time.time() - t0) / 60

vram_peak = torch.cuda.max_memory_reserved() / 1e9

print()
print("=" * 60)
print("TRAINING COMPLETE")
print("=" * 60)
print(f"  Steps          : {train_result.global_step:,}")
print(f"  Training loss  : {train_result.training_loss:.4f}")
print(f"  Elapsed        : {elapsed_min:.1f} min")
print(f"  Peak VRAM      : {vram_peak:.1f} GB")
print()
print("V2 reference:")
print("  Steps: 1,887  |  Loss: 1.5058  |  Time: 144.0 min  |  VRAM: 15.5 GB")
print()
print("V1 reference (1 epoch, Q&A format):")
print("  Steps: 321    |  Loss: 2.4400  |  Time:  23.3 min  |  VRAM: 13.4 GB")

# 136m 52.3s

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,867 | Num Epochs = 3 | Total steps = 1,827
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 128,450,560 of 14,896,757,760 (0.86% trained)


VRAM before training: 11.7 GB
Starting training — 3 epochs, ~1,825 gradient updates...

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
25,2.719300
50,2.515000
75,2.466600
100,2.408600
125,2.407400
150,2.359900
175,2.320600
200,2.282900
225,2.288200
250,2.396300



TRAINING COMPLETE
  Steps          : 1,827
  Training loss  : 1.5258
  Elapsed        : 136.9 min
  Peak VRAM      : 15.3 GB

V2 reference:
  Steps: 1,887  |  Loss: 1.5058  |  Time: 144.0 min  |  VRAM: 15.5 GB

V1 reference (1 epoch, Q&A format):
  Steps: 321    |  Loss: 2.4400  |  Time:  23.3 min  |  VRAM: 13.4 GB


---
# Part 6: Save the LoRA Adapter

We save only the LoRA adapter weights — not the full 14B model.  This keeps
the checkpoint small (~520 MB for r=32 vs ~28 GB for a full model copy).
At inference time Unsloth loads the base model from HuggingFace Hub and
merges the adapter on-the-fly.

**V3 output path**: `outputs/qwen3_14b_peterson_v3_lora/`

The V2 adapter at `outputs/qwen3_14b_peterson_v2_lora/` is **not touched**.
Both adapters coexist and can be compared independently in the
`AllModels_JordanPeterson_Comparison.ipynb` notebook.

In [9]:
print(f"Saving V3 LoRA adapter to {OUTPUT_DIR} ...")

model.save_pretrained(str(OUTPUT_DIR))
tokenizer.save_pretrained(str(OUTPUT_DIR))

# List saved files with sizes
adapter_files = (
    list(OUTPUT_DIR.glob("*.safetensors"))
    + list(OUTPUT_DIR.glob("*.bin"))
)
total_mb = sum(f.stat().st_size for f in adapter_files) / 1e6

print()
print("Adapter files:")
for f in sorted(adapter_files):
    print(f"  {f.name}  ({f.stat().st_size/1e6:.1f} MB)")

print()
print(f"Total adapter size: {total_mb:.1f} MB")
print(f"  (V2 r=32 was 513.9 MB; V3 r=32 should be similar)")
print()
print(f"V2 adapter (untouched) : outputs/qwen3_14b_peterson_v2_lora/")
print(f"V3 adapter (just saved): {OUTPUT_DIR}")

Saving V3 LoRA adapter to outputs/qwen3_14b_peterson_v3_lora ...

Adapter files:
  adapter_model.safetensors  (513.9 MB)

Total adapter size: 513.9 MB
  (V2 r=32 was 513.9 MB; V3 r=32 should be similar)

V2 adapter (untouched) : outputs/qwen3_14b_peterson_v2_lora/
V3 adapter (just saved): outputs/qwen3_14b_peterson_v3_lora


---
# Part 7: Inference Test

The same 5 evaluation prompts are used here as in V1 and V2 so results are
directly comparable.  Copy-paste the outputs into a comparison doc alongside
V2 outputs to spot-check whether the front-matter fix produces visibly better
responses (especially for Maps of Meaning questions).

We run greedy decoding (`do_sample=False`) for deterministic, reproducible
outputs — the same setting used in all comparison notebooks.

In [10]:
FastLanguageModel.for_inference(model)

EVAL_PROMPTS = [
    "What is the relationship between order and chaos in human experience?",
    "Why is personal responsibility the foundation of a meaningful life?",
    "How do ancient myths and stories reveal truths about human nature?",
    "What does it mean to pursue what is meaningful rather than what is expedient?",
    "How should a person confront suffering rather than flee from it?",
]


def ask_v3(question: str, max_new_tokens: int = 300) -> str:
    '''
    Generate a response from the V3 fine-tuned model.

    Uses greedy decoding (do_sample=False) for reproducibility, consistent
    with how V1 and V2 outputs are evaluated in the comparison notebooks.
    enable_thinking=False keeps reasoning mode disabled, matching training.
    '''
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": question},
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )
    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens     = max_new_tokens,
            do_sample          = False,
            temperature        = 1.0,
            repetition_penalty = 1.1,
        )

    new_tokens = out[0][inputs["input_ids"].shape[1]:]
    response   = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    # Strip any residual thinking blocks
    response   = re.sub(r"<think>.*?</think>", "", response, flags=re.DOTALL).strip()
    return response


print("Testing V3 fine-tuned model (greedy decoding)...")
print()
for i, prompt in enumerate(EVAL_PROMPTS):
    print("-" * 70)
    print(f"Q{i+1}: {prompt}")
    print("-" * 70)
    answer = ask_v3(prompt)
    print(answer if answer.strip() else "(empty response)")
    print()

Testing V3 fine-tuned model (greedy decoding)...

----------------------------------------------------------------------
Q1: What is the relationship between order and chaos in human experience?
----------------------------------------------------------------------
of the world parents (and their “children”) from a different perspective. The great mother—nature, in her benevolent guise—provides us with the necessities and opportunities of life: she is the source of rich harvest and the giver of freely available water. She eternally nurtures and protects. The terrible mother—nature, in revolutionary guise—is the sudden appearance of the stranger; the force that destroys crops and livestock through drought, flood or pestilence; the giver of disease and death. She is the eternal source of threat to the individual state and culture, and can manifest herself in unexpected ways. It is the combination of these two eternally extant “personality aspects” that constitute the most primordial deit

---
# Conclusions

## What V3 Changed vs V2

V3 uses identical hyperparameters to V2 (r=32, 3 epochs, 2e-4 LR).  The
only change is the training data: the DataPrep notebook's front-matter removal
heuristic is now applied, removing publisher metadata pages from all four books.

The net effect: **162 fewer Q&A pairs** (5,029 → 4,867), all subtracted from
boilerplate passages.  No philosophical content was removed.

## Expected Improvement

The primary expected improvement is in **Maps of Meaning responses**.  V2 had
~57 passages (1,826 - ~1,769 actual content passages) drawn from publisher
pages, title pages, and the table of contents.  V3 eliminates those.  If the
model still produces responses that begin with book metadata, check the DataPrep
notebook's front-matter detection for that book.

For the other three books, response quality should be indistinguishable from V2
— the data change was minimal.

## Adding V3 to the AllModels Comparison Notebook

To compare V3 against V1, V2, and the base models:

1. Open `AllModels_JordanPeterson_Comparison.ipynb`
2. Add `"qwen3_v3"` to `MODEL_KEYS`
3. Add `"qwen3_v3": "./outputs/qwen3_14b_peterson_v3_lora/"` to `MODEL_PATHS`
4. Add a colour entry in `MODEL_COLORS`
5. Delete `comparison_cache_all_models/qwen3_v3_results.pkl` if it exists
6. Re-run the comparison notebook

## Next Levers If Further Improvement Is Needed

| Option | Expected gain | Effort |
|--------|--------------|--------|
| 5 epochs (instead of 3) | Marginal style improvement; risk of overfitting | Low |
| 3 questions per passage | More data diversity, ~7,400 pairs | Medium |
| `r=64, alpha=64` | More adapter capacity; +200 MB checkpoint | Low |
| Human-edited answers | Highest quality ceiling | High |
| Qwen3-32B (if it fits) | Stronger base model | High |

In [11]:
!nvidia-smi

Sat Feb 21 12:10:17 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.126.09             Driver Version: 580.126.09     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:04:00.0 Off |                  Off |
| 39%   50C    P8             26W /  450W |   12648MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----